In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style( 'whitegrid' )

# Parameters

In [3]:
year_start = 'September 1'

In [4]:
data_fp = './data/exports/News_Report_2023-07-25.csv'

In [5]:
groupings = [ 'Research Topics', 'Press Types', 'Categories' ]

# Preprocessing

In [ ]:
# Load data
df = pd.read_csv( data_fp, parse_dates=[ 'Date', ] )

In [ ]:
# Drop drafts
df.drop( df.index[df['Date'].dt.year == 1970], axis='rows', inplace=True )

In [ ]:
# Drop weird articles---ancient ones w/o a title or press type
df.dropna( axis='rows', how='any', subset=[ 'Title', 'Press Types', ], inplace=True )

In [ ]:
# Get rid of HTML ampersands
for str_column in [ 'Title', 'Research Topics', 'Categories' ]:
    df[str_column] = df[str_column].str.replace( '&amp;', '&' )

In [ ]:
# Get date bins
start_year = df['Date'].min().year - 1
end_year = df['Date'].max().year + 1
date_bins = pd.date_range(
    '{} {}'.format( year_start, start_year ),
    pd.Timestamp.now() + pd.offsets.DateOffset( years=1 ),
    freq = pd.offsets.DateOffset( years=1 ),
)
date_bin_labels = date_bins.year[:-1]

In [ ]:
# Add the year published (using the above start date )
df['Year'] = pd.cut( df['Date'], date_bins, labels=date_bin_labels ) 

# Visualize

In [ ]:
for group_by_i in groupings:
    df_i = df.copy()

    # Explode and group
    df_i[group_by_i] = df_i[group_by_i].str.split( '|' )
    df_i = df_i.explode( group_by_i )

    counts = df_i.pivot_table( index='Year', columns=group_by_i, values='id', aggfunc='count' )

    years = counts.index.astype( int )

    facet_grid = sns.relplot(
        counts,
        kind = 'line',
        dashes = False,
        linewidth = 3,
        aspect = 2
    )
    facet_grid.ax.set_xlim( years[0], years[-1] )
    facet_grid.ax.set_ylim( 0, facet_grid.ax.get_ylim()[1] )
    ticks = facet_grid.ax.set_xticks( years )
    facet_grid.ax.set_ylabel( 'Count' )

    save_fp = './figures/count_per_year.{}.pdf'.format( group_by_i.lower().replace( ' ', '_' ) )

    facet_grid.fig.savefig( save_fp, bbox_inches='tight' )

# Scratch

In [17]:
df = pd.read_csv( '../test/test_data/press_office.csv' )

In [18]:
df.drop( 'Unnamed: 0', axis='columns', inplace=True )

In [21]:
df['Title (optional)'] = df['Title (optional)'].str.strip()

In [10]:
df.drop( 'Unnamed: 5', axis='columns', inplace=True )

df.dropna( axis='rows', inplace=True )

df = df.set_axis( df.columns.str.strip(), axis='columns' )

for column in df.columns:
    try:
        df[column] = df[column].str.strip()
    except AttributeError:
        continue

for column in df.columns:
    try:
        df[column] = df[column].values.astype( int )
    except ValueError:
        continue

df.to_csv(  '../test/test_data/press_office.csv' )

df.to_xml

# pd.DataFrame.to_excel(

SyntaxError: incomplete input (2071547193.py, line 23)

In [14]:
df.set_index( 'id', inplace=True )

In [13]:
pd.DataFrame.to_excel?

Signature:
pd.DataFrame.to_excel(
    self,
    excel_writer,
    sheet_name: 'str' = 'Sheet1',
    na_rep: 'str' = '',
    float_format: 'str | None' = None,
    columns: 'Sequence[Hashable] | None' = None,
    header: 'Sequence[Hashable] | bool_t' = True,
    index: 'bool_t' = True,
    index_label: 'IndexLabel' = None,
    startrow: 'int' = 0,
    startcol: 'int' = 0,
    engine: 'str | None' = None,
    merge_cells: 'bool_t' = True,
    encoding: 'lib.NoDefault' = <no_default>,
    inf_rep: 'str' = 'inf',
    verbose: 'lib.NoDefault' = <no_default>,
    freeze_panes: 'tuple[int, int] | None' = None,
    storage_options: 'StorageOptions' = None,
) -> 'None'
Docstring:
Write object to an Excel sheet.

To write a single object to an Excel .xlsx file it is only necessary to
specify a target file name. To write to multiple sheets it is necessary to
create an `ExcelWriter` object with a target file name, and specify a sheet
in the file to write to.

Multiple sheets may be written to by s

In [16]:
df.to_excel( '../test/test_data/press_office.xlsx', )